# AVA

Load paths

In [ ]:
import json
import os
with open('../paths.json', 'r') as f:
    paths = json.load(f)
    pdownloads = paths["paths"][paths["cloud"]]["downloads"]
    pdatasets = paths["paths"][paths["cloud"]]["datasets"]

In [ ]:
if(not(os.path.isdir(pdownloads))):
    !mkdir $pdownloads
if(not(os.path.isdir(pdatasets))):
    !mkdir $pdatasets
    !mkdir $pdatasets/ava
    !mkdir $pdatasets/ava/labels
    !mkdir $pdatasets/ava/images

## Download AVA dataset via rtorrent

In [ ]:
!wget http://academictorrents.com/download/71631f83b11d3d79d8f84efe0a7e12f0ac001460.torrent -P $pdownloads

Run this command directly in a bash session

In [ ]:
!cd $pdownloads ; sudo rtorrent 71631f83b11d3d79d8f84efe0a7e12f0ac001460.torrent

In [ ]:
!cat $pdownloads/AVA_dataset/README.txt

## Database extraction

Run this command directly in a bash session

In [ ]:
!cd $pdatasets/ava/images ; sudo 7z e $pdownloads/AVA_dataset/images/images.7z.001 -aos > ../7z.log

Monitor extraction

In [ ]:
!ps -aux | grep 7z

In [ ]:
!cat ../datasets/ava/7z.log | tail -2

In [ ]:
!ls ../datasets/ava/images | wc -l

## Check for missing images & download them

In [ ]:
import json
import os
with open('ava labels/ava_labels_test.json', 'r') as f:
    test_json = json.load(f)
with open(pdownloads+'/ava_labels_train.json', 'r') as f:
    train_json = json.load(f)

In [ ]:
i = 0
for iid in test_json['image_id']:
    name = test_json['image_id'][iid]
    file = name[0: -4]
    if(not(os.path.isfile(pdatasets+'/ava/images/' + name))):
        !wget https:$(wget -O - https://www.dpchallenge.com/image.php?IMAGE_ID=$file | grep -o "//images\.dpchallenge\.com/images_challenge/[0-9]*-[0-9]*/[0-9]*/[0-9]*/Copyrighted_Image_Reuse_Prohibited_$name")
        !mv Copyrighted_Image_Reuse_Prohibited_$name $pdatasets/ava/images/$name
        i = i + 1
print(i)

In [ ]:
i = 0
for iid in train_json['image_id']:
    name = train_json['image_id'][iid]
    file = name[0: -4]
    if(not(os.path.isfile(pdatasets+'/ava/images/' + name))):
        !wget https:$(wget -O - https://www.dpchallenge.com/image.php?IMAGE_ID=$file | grep -o "//images\.dpchallenge\.com/images_challenge/[0-9]*-[0-9]*/[0-9]*/[0-9]*/Copyrighted_Image_Reuse_Prohibited_$name")
        !mv Copyrighted_Image_Reuse_Prohibited_$name $pdatasets/ava/images/$name
        i = i + 1
print(i)

Check for 0 sized files, to re-download them

In [ ]:
!find $pdatasets/ava/images -size 0 -print

Manually remove them and re-download them

In [ ]:
file = '220902'
name = file + '.jpg'
!rm $pdatasets/ava/images/$name
!wget https:$(wget -O - https://www.dpchallenge.com/image.php?IMAGE_ID=$file | grep -o "//images\.dpchallenge\.com/images_challenge/[0-9]*-[0-9]*/[0-9]*/[0-9]*/Copyrighted_Image_Reuse_Prohibited_$name")
!mv Copyrighted_Image_Reuse_Prohibited_$name $pdatasets/ava/images/$name


Check for files that cannot be opened by PIL

In [ ]:
import pandas as pd
train_df = pd.read_json('ava labels/train.json')
test_df = pd.read_json('ava labels/test.json')
df = pd.concat([train_df, test_df], ignore_index=True)

failed = []
i = 0
f = 0
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for x in df["image_id"]:
    i = i + 1
    try:
        img = Image.open(pdatasets+'/ava/images/' + x, 'r')
    except:
        failed.append(x)
        f = f + 1
    print("processed " + str(i) + ", failed " + str(f))

In [ ]:
print(failed)